In [2]:
using Revise
using DataStructures
using Distributions
using Quarton

### Example: Network of queues with non-probabilistic routing

Page 17. This represents disk usage for a job.

 1. Jobs arrive at an exponential rate.
 2. A CPU handles those jobs and feeds them to the queue for Disk 1.
 3. Disk 1 processes a job and either sends it to the sink or to Disk 2.
    a. If the job has a mark of 1, it goes to Disk 2 and gets incremented.
    b. If the job has a mark of 2, it goes to the sink.
 4. Disk 2 processes jobs and returns them to the queue of Disk 1.


In [5]:
function nonprobabilistic_routing()
    arrival_rate = 1.0
    service_rate = 1.0
    model = QueueModel()
    token_builder = (when, rng) -> CountedWork(1.0, when)
    source = add_queue!(model, InfiniteSourceQueue())
    CPU_queue = add_queue!(model, FIFOQueue())
    disk1_queue = add_queue!(model, FIFOQueue())
    disk2_queue = add_queue!(model, FIFOQueue())
    sink = add_queue!(model, SinkQueue())
    arrival = add_server!(model, ArrivalServer(arrival_rate))
    CPU = add_server!(model, ModifyServer(service_rate))
    on_output = token -> (token.mark = 2; nothing)
    assign_by_mark = SizeIntervalAssignment(t -> (t.mark == 1) ? :around : :out)
    disk1 = ModifyServer(
        service_rate, disbursement=assign_by_mark, modify=on_output
        )
    add_server!(model, disk1)
    disk2 = add_server!(model, ModifyServer(service_rate))
    connect!(model, source, arrival, :only)
    connect!(model, arrival, CPU_queue, :only)
    connect!(model, CPU_queue, CPU, :only)
    connect!(model, CPU, sink, :only)
    connect!(model, disk1_queue, disk1, :only)
    connect!(model, disk1, sink, :out)
    connect!(model, disk1, disk2_queue, :around)
    connect!(model, disk2_queue, disk2, :only)
    connect!(model, disk2, disk1_queue, :only)
    # connect!(model, CPU, sink, :only)
    check_model(model)
    trajectory = Trajectory(2342334)
    start_time = zero(Float64)
    activate!(model, trajectory, arrival, CountedWork())
    when = 0.0
    while when < 10000.0
        when, which = next(trajectory)
        step!(model, trajectory, (when, which))
    end
    response = sink.retire_total_duration / sink.retire_cnt
    return response
end
nonprobabilistic_routing()

24.297218383984642

### Example: Finite buffer

This buffer will drop jobs when it is full. This is on page 17.

In [17]:
function finite_buffer(arrival_rate, service_rate, keep_cnt)
    model = QueueModel()
    source = add_queue!(model, InfiniteSourceQueue())
    fifo = add_queue!(model, FiniteFIFOQueue(keep_cnt))
    sink = add_queue!(model, SinkQueue())
    s1 = add_server!(model, ArrivalServer(arrival_rate))
    s2 = add_server!(model, ModifyServer(service_rate))
    connect!(model, source, s1, :only)
    connect!(model, s1, fifo, :only)
    connect!(model, fifo, s2, :only)
    connect!(model, s2, sink, :only)
    check_model(model)
    trajectory = Trajectory(2342334)
    start_time = zero(Float64)
    activate!(model, trajectory, s1, Work())
    when = 0.0
    while when < 10000.0
        when, which = next(trajectory)
        step!(model, trajectory, (when, which))
    end
    response = sink.retire_total_duration / sink.retire_cnt
    drops = fifo.drop_cnt
    return response, drops
end
# How many drops do we get when the rate of arrivals equals the rate
# of the server, over the 10,000s?
keep_cnt = 9
finite_buffer(1.0, 1.0, keep_cnt)

(6.037175250004293, 943)